# Monitoring of model
 Evaluate model on data and obtain metrics of performance

## Load libraries

In [67]:
pip install tabgan==1.3.3

Note: you may need to restart the kernel to use updated packages.


In [68]:
!pip install lightgbm

In [69]:
pip install scikit-plot

Note: you may need to restart the kernel to use updated packages.


In [70]:
%pip install scipy==1.11.4

Note: you may need to restart the kernel to use updated packages.


In [71]:
pip install --upgrade pandas "dask[complete]"

Note: you may need to restart the kernel to use updated packages.


In [72]:
## librerias requeridas
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import re
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_lift_curve
from scikitplot.helpers import binary_ks_curve 
from scikitplot.metrics import plot_ks_statistic
from scikitplot.helpers import cumulative_gain_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from datetime import datetime, timedelta

import pyarrow as pa
import pyarrow.parquet as pq

In [73]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.memory_usage', True)

## Load model and data to evaluate

Load variables names from file

In [74]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/viamericas-dev/Analytics/5.Fraud'

In [75]:
import os
import joblib
# load model
#modelo = joblib.load("ModeloV2TodosMenosVariables.pkl") 
#variables = open('./ModeloV2TodosMenosVariables.txt','r')
modelo = joblib.load("ModeloBite.pkl") 
variables = open('./ModeloBite.txt','r')

In [76]:
modelo

LGBMClassifier(colsample_bytree=0.84, importance_type='gini',
               learning_rate=0.07, max_depth=20, min_child_weight=0.0026,
               n_estimators=520, num_leaves=30, objective='binary',
               scale_pos_weight=0.003, subsample=1)

## Veo si puedo sacar los nombres directo del modelo q cargue
## Veo como cargar el punto de corte

In [77]:
variables = open('./ModeloBite.txt','r')
var_input = variables.read()

#1er prueba
var_input = ['branch_minutes_since_last_transaction',
    'branch_trans_3m',  'branch_has_fraud',  
    'branch_trans_40min', 'branch_trans_10min', 'cash_pick_up_40min', 
    'location_nro_fraud', 'sender_trans_3m', 
    'sender_nro_fraud','01_isMexico', 
    '01_sender_branch_state', '01_var_range_hist', 
    '01_branch_fraud', '01_location_fraud', '01_sender_fraud', 
    '01_isCashPick', '01_isBankDep', 
    'sender_days_to_last_transaction_more7m', 
    '01_receiver_transaction_count','01_sender_sending_days',
    '01_branch_working_days', '01_net_amount_receiver', 
    '01_sender_minutes_since_last_transaction_2days', 
    '01_sender_days_to_last_transaction_365', 
    '01_sender_days_to_last_transaction_7m', '01_hour_receiver']

In [78]:
#2da prueba
var_input = ['branch_minutes_since_last_transaction',
    'branch_trans_3m',  'branch_has_fraud',  
    'branch_trans_40min', 'branch_trans_10min', 'cash_pick_up_40min', 
    'location_nro_fraud', 'sender_trans_3m', 
    'sender_nro_fraud','01_isMexico', 
    '01_sender_branch_state', '01_var_range_hist', 
    '01_branch_fraud', '01_location_fraud',
    '01_isCashPick', '01_isBankDep', 
    'sender_days_to_last_transaction_more7m', 
    'receiver_transaction_count','01_sender_sending_days',
    'branch_working_days', '01_net_amount_receiver', 
    'sender_minutes_since_last_transaction_2days', 
    'sender_days_to_last_transaction_365', 
    'sender_days_to_last_transaction_7m', '01_hour_receiver']

In [79]:
s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos"
dfTrain = pd.read_parquet(f"s3://{s3_url}/Train/Train.parquet")
tgTrain = dfTrain['target']

dfVal = pd.read_parquet(f"s3://{s3_url}/Validation/Validation.parquet")
tgVal = dfVal['target']

#Uni test y Val
#X = pd.concat([dfTrain[var_input], dfVal[var_input]], ignore_index=True) 
X = pd.concat([dfTrain, dfVal], ignore_index=True) 
Y = pd.DataFrame(pd.concat([dfTrain['target'], dfVal['target']], ignore_index=True)) 

dias_target = ['day_name_receiver_0', 'day_name_receiver_1',
       'day_name_receiver_2', 'day_name_receiver_3',
       'day_name_receiver_4', 'day_name_receiver_5',
       'day_name_receiver_6','target']

testeo =  pd.concat([X, Y], axis=1)
Y_target = pd.DataFrame(pd.concat([dfTrain[dias_target], dfVal[dias_target]], ignore_index=True)) 

In [80]:
# Uso esto para sacar el punto de corte. Cuando lo pueda guardar esta celda se va
# Prediciendo la variable objetivo para el conjunto de datos de prueba 'X_TESTEO' utilizando el clasificador LightGBM
testeo_data_pred_rf = modelo.predict(testeo[var_input])

# Prediciendo las probabilidades de clase para cada observación en 'X_TESTEO' utilizando el clasificador LightGBM
probab_rf = modelo.predict_proba(testeo[var_input])

# Eliminando la primera columna (se asume que son las probabilidades de la clase negativa) de las probabilidades predichas
# Se asume que la segunda columna contiene las probabilidades para la clase positiva
score_rf = np.delete(probab_rf, np.s_[0], axis=1)

# Creando una copia del DataFrame 'Y_c' para preservar los datos originales
Y_c = Y_target.copy()

# Agregando una nueva columna 'preds_rf' al DataFrame copiado con las predicciones del clasificador LightGBM
Y_c['preds_rf'] = testeo_data_pred_rf

# Agregando una nueva columna 'score_rf' al DataFrame copiado con las probabilidades de la clase positiva
Y_c['score_rf'] = score_rf

#Calculo el primer percentil para sacar el punto de corte
percentiles = pd.qcut(Y_c['score_rf'], q=1000, duplicates='drop')
bins = percentiles
percentile_label = {p: l for l, p in enumerate(sorted(percentiles.unique(), reverse=True), start=1)}
percentiles = percentiles.map(percentile_label)
Y_c['FRAUD_DECILE'] = np.nan
Y_c['FRAUD_DECILE'] = percentiles
Y_c['FRAUD_DECILE'] = Y_c['FRAUD_DECILE'].astype('Int32')

#saco el punto de corte del primer centil para aplicarlo al test
corte_01 = Y_c.score_rf[Y_c['FRAUD_DECILE']==1].min()
corte_02 = Y_c.score_rf[Y_c['FRAUD_DECILE']==2].min()
corte_03 = Y_c.score_rf[Y_c['FRAUD_DECILE']==3].min()
corte_04 = Y_c.score_rf[Y_c['FRAUD_DECILE']==4].min()
corte = Y_c.score_rf[Y_c['FRAUD_DECILE']==10].min()

In [81]:
bins_labels = percentile_label

In [82]:
Y_c['pred_corte'] = Y_c['score_rf']>corte_01
pd.DataFrame(confusion_matrix(Y_c['target'], Y_c['pred_corte']))

,0,1
0,16708132,16141
1,12,583


In [83]:
s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos"
tes = pd.read_parquet(f"s3://{s3_url}/Test/Test.parquet")

testeo = pd.concat([X, tes], axis = 0, ignore_index=True)
Y_target = pd.DataFrame(pd.concat([Y_target, tes[dias_target]], ignore_index=True)) 

In [84]:
s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos"
tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Data2024.parquet")

testeo = pd.concat([testeo, tes], axis = 0, ignore_index=True)
Y_target = pd.DataFrame(pd.concat([Y_target, tes[dias_target]], ignore_index=True)) 

## Apply model

In [85]:
# Prediciendo la variable objetivo para el conjunto de datos de prueba 'X_TESTEO' utilizando el clasificador LightGBM
testeo_data_pred_rf = modelo.predict(testeo[var_input])

# Prediciendo las probabilidades de clase para cada observación en 'X_TESTEO' utilizando el clasificador LightGBM
probab_rf = modelo.predict_proba(testeo[var_input])

# Eliminando la primera columna (se asume que son las probabilidades de la clase negativa) de las probabilidades predichas
# Se asume que la segunda columna contiene las probabilidades para la clase positiva
score_rf = np.delete(probab_rf, np.s_[0], axis=1)


In [86]:
# Creando una copia del DataFrame 'Y_TESTEO' para preservar los datos originales
Y_c = Y_target.copy()

# Agregando una nueva columna 'preds_rf' al DataFrame copiado con las predicciones del clasificador LightGBM
Y_c['preds_rf'] = testeo_data_pred_rf

# Agregando una nueva columna 'score_rf' al DataFrame copiado con las probabilidades de la clase positiva
Y_c['score_rf'] = score_rf

In [87]:
pd.DataFrame(confusion_matrix(Y_c['target'], Y_c['preds_rf']))

,0,1
0,32966451,20
1,315,503


In [88]:
Y_c['pred_corte'] = Y_c['score_rf']>corte
pd.DataFrame(confusion_matrix(Y_c['target'], Y_c['pred_corte']))

,0,1
0,32637833,328638
1,39,779


In [89]:
# Dividir las predicciones en deciles y calcular la captura del 30% en el conjunto de entrenamiento
percentiles = pd.qcut(Y_c['score_rf'], q=100, duplicates='drop')
percentile_label = {p: l for l, p in enumerate(sorted(percentiles.unique(), reverse=True), start=1)}
percentiles = percentiles.map(percentile_label)
Y_c['FRAUD_DECILE'] = np.nan
Y_c['FRAUD_DECILE'] = percentiles
Y_c['FRAUD_DECILE'] = Y_c['FRAUD_DECILE'].astype('Int32')
print('FRAUD CAPTURA 30%:')
print(sum(Y_c[Y_c['FRAUD_DECILE'] < 4]['target']) / sum(Y_c['target']))

FRAUD CAPTURA 30%:
0.9743276283618582


In [90]:
DECILES = pd.crosstab(Y_c['FRAUD_DECILE'], Y_c['target'])
DECILES

target,False,True
FRAUD_DECILE,,
1,328894,779
2,329657,16
3,329671,2
4,329670,3
5,329670,3
...,...,...
96,329673,0
97,329673,0
98,329673,0


In [91]:
Y_c[['score_rf','FRAUD_DECILE']].groupby(['FRAUD_DECILE']).agg({'score_rf': [ 'min', 'max']}) 

score_rf                  
                           min               max
FRAUD_DECILE                                    
1            0.000000017561167 0.999999984219915
2            0.000000005069150 0.000000017561136
3            0.000000002493566 0.000000005069130
4            0.000000001514816 0.000000002493562
5            0.000000001029196 0.000000001514807
...                        ...               ...
96           0.000000000000350 0.000000000000424
97           0.000000000000277 0.000000000000350
98           0.000000000000203 0.000000000000277
99           0.000000000000126 0.000000000000203
100          0.000000000000002 0.000000000000126

[100 rows x 2 columns]

## Analizo distintos puntos de corte

In [92]:
testeo['date_receiver'].dt.month
testeo['year'] = testeo['date_receiver'].dt.year
Y_c['fraud_classification'] = testeo['fraud_classification']
Y_c['targetF'] = (testeo['fraud_classification']=='Real')

In [93]:
#Veo cuantos falsos positivos tengo con el punto de corte
Y_c['pred_corte_01'] = Y_c['score_rf']>=corte_01
tes_corte_01 = testeo[Y_c['pred_corte_01']==True]
Y_c_corte_01 = Y_c[Y_c['pred_corte_01']==True]

pd.crosstab(tes_corte_01['mes'], Y_c_corte_01['targetF'])

targetF,False,True
mes,,
1,3583,78
2,3800,38
3,6316,23
4,2095,61
5,2122,45
6,2180,66
7,2130,30
8,2205,37
9,2469,28


In [94]:
#Veo cuantos falsos positivos tengo con el punto de corte
Y_c['pred_corte_02'] = Y_c['score_rf']>corte_02
tes_corte_02 = testeo[Y_c['pred_corte_02']==True]
Y_c_corte_02 = Y_c[Y_c['pred_corte_02']==True]

pd.crosstab(tes_corte_02['mes'], Y_c_corte_02['targetF'])

targetF,False,True
mes,,
1,7560,80
2,7604,39
3,11146,25
4,4185,61
5,4232,45
6,4296,67
7,4342,30
8,4464,37
9,4806,31


In [95]:
#Veo cuantos falsos positivos tengo con el punto de corte
Y_c['pred_corte_03'] = Y_c['score_rf']>corte_03
tes_corte_03 = testeo[Y_c['pred_corte_03']==True]
Y_c_corte_03 = Y_c[Y_c['pred_corte_03']==True]

pd.crosstab(tes_corte_03['mes'], Y_c_corte_03['targetF'])

targetF,False,True
mes,,
1,11486,82
2,11510,39
3,15654,25
4,6251,61
5,6337,45
6,6468,67
7,6580,30
8,6637,37
9,7174,33


In [96]:
#Veo cuantos falsos positivos tengo con el punto de corte
Y_c['pred_corte_04'] = Y_c['score_rf']>corte_04
tes_corte_04 = testeo[Y_c['pred_corte_04']==True]
Y_c_corte_04 = Y_c[Y_c['pred_corte_04']==True]

pd.crosstab(tes_corte_04['mes'], Y_c_corte_04['targetF'])

targetF,False,True
mes,,
1,15287,83
2,15395,39
3,20198,27
4,8425,61
5,8421,45
6,8643,67
7,8770,30
8,8859,37
9,9511,34


In [112]:
#Veo cuantos falsos positivos tengo con el punto de corte
tes_corte = testeo[Y_c['pred_corte']==True]
Y_c_corte = Y_c[Y_c['pred_corte']==True]

pd.crosstab(tes_corte['mes'], Y_c_corte['target'])

target,False,True
mes,,
1,37929,140
2,38426,68
3,46385,53
4,20965,86
5,21484,69
6,22073,99
7,22060,50
8,21910,68
9,23892,66


In [114]:
#Bajo fraudes diarios para comparacion
testigo = testeo['date_receiver'].between('2024-01-01','2024-03-31')
X_testigo = testeo[testigo]
Y_c_testigo = Y_c[testigo]

X_testigo_corte = X_testigo[Y_c_testigo['pred_corte_01']==True]
Y_c_testigo_corte = Y_c_testigo[Y_c_testigo['pred_corte_01']==True]

ALERTAS_2024 = pd.crosstab(X_testigo_corte['date_receiver'].dt.floor('d'), Y_c_testigo_corte['targetF'])
ALERTAS_2024.to_csv ('./alertas2024.csv')

## Transformo los datos para subirlos a Athena

In [98]:
Buckets = pd.read_csv('./BucketsModeloBite.csv')

In [99]:
# Create bins from reference dataframe
pd.options.display.float_format = "{:,.15f}".format
# Convert data types of 'Min_Value' and 'Max_Value' to numeric
Buckets['min'] = pd.to_numeric(Buckets['min'])
Buckets['max'] = pd.to_numeric(Buckets['max'])
Y_c['score_rf'] = pd.to_numeric(Y_c['score_rf'])

#bins = np.sort(np.append(Buckets['min'], Buckets['max'].iloc[-1]))
bins = np.sort(np.append(Buckets['min'], np.inf))  

labels = np.sort(Buckets['final_score'])
#labels = np.sort(1000-Buckets['FRAUD_DECILE'])

# Recode scores using pd.cut()
Y_c['score_AWS'] = pd.cut(Y_c['score_rf'], bins=bins, labels=labels) ##, include_lowest=True, right=False)
#Y_c['Recode'] = Y_c['Recode'].astype('Int32')

Y_c.head()

,day_name_receiver_0,day_name_receiver_1,day_name_receiver_2,day_name_receiver_3,day_name_receiver_4,day_name_receiver_5,day_name_receiver_6,target,preds_rf,score_rf,pred_corte,FRAUD_DECILE,fraud_classification,targetF,pred_corte_01,pred_corte_02,pred_corte_03,pred_corte_04,score_AWS
0,False,False,True,False,False,False,False,False,False,0.000000000076448,False,21,None,False,False,False,False,False,0.788000000000000
1,False,False,False,False,False,True,False,False,False,0.000000000000620,False,93,None,False,False,False,False,False,0.074000000000000
2,False,False,False,False,False,True,False,False,False,0.000000000000721,False,92,None,False,False,False,False,False,0.086000000000000
3,True,False,False,False,False,False,False,False,False,0.000000000010735,False,50,None,False,False,False,False,False,0.502000000000000
4,False,False,False,False,False,True,False,False,False,0.000000000264493,False,11,None,False,False,False,False,False,0.890000000000000


In [100]:
Y_c['pred_corte_01'].value_counts() #34841

pred_corte_01
False    32932424
True        34865
Name: count, dtype: int64

In [101]:
Y_c.loc[(Y_c['score_AWS']==998) & (Y_c['pred_corte_01']==True),'score_rf']

Series([], Name: score_rf, dtype: float64)

In [102]:
pd.DataFrame(Y_c.loc[Y_c['pred_corte_01']==True,'score_AWS'].value_counts())

,count
score_AWS,
0.999000000000000,34864
0.998000000000000,1
0.671000000000000,0
0.658000000000000,0
0.659000000000000,0
...,...
0.338000000000000,0
0.339000000000000,0
0.340000000000000,0


In [103]:
pd.DataFrame(Y_c.loc[Y_c['target']==True,'pred_corte_01'].value_counts())

,count
pred_corte_01,
True,704
False,114


In [104]:
corte_01
9.922548639402274e-07

9.922548639402274e-07

In [105]:
print('max ',Y_c.loc[(Y_c['pred_corte_01']==True) & (Y_c['score_AWS']==998),'score_rf'].max())
print('min ',Y_c.loc[(Y_c['pred_corte_01']==True) & (Y_c['score_AWS']==998),'score_rf'].min())


max  nan
min  nan


In [109]:
df = pd.concat([testeo[['id_branch','id_receiver']], Y_c[['score_AWS','pred_corte_01']]], axis=1) 
df = df.rename(columns={"pred_corte_01": "was_fraud_AWS"})

## Guardo en Athena

In [107]:
pip install awswrangler

Note: you may need to restart the kernel to use updated packages.


In [110]:
import awswrangler as wr
import pandas as pd
import boto3
import time
from time import sleep

# Reemplazar aca por el dataframe a guardar
df = df

wr.s3.to_parquet(
    df=df,
    path='s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ml_score_aws2/',
    dataset=True,
    mode="overwrite"
)

#lambda_crawler
def wait_for_crawler_completion(crawler_name: str, poll_interval: int = 15) -> str:
	"""
	Waits until Glue crawler completes and
	returns the status of the latest crawl run.
	Raises AirflowException if the crawler fails or is cancelled.
	:param crawler_name: unique crawler name per AWS account
	:param poll_interval: Time (in seconds) to wait between two consecutive calls to check crawler status
	:return: Crawler's status
	"""
	client = boto3.client('glue')
	response = client.start_crawler(Name=crawler_name)
	failed_status = ['FAILED', 'CANCELLED']
	sleep(5)

	while True:
		crawler = client.get_crawler(Name=crawler_name)
		crawler_state = crawler['Crawler']['State']
		if 'LastCrawl' in crawler['Crawler']:
			if crawler_state == 'STOPPING' or crawler_state == 'READY':
				print("State: %s", crawler_state)
				print("crawler_config: %s", crawler)
				crawler_status = crawler['Crawler']['LastCrawl']['Status']
				if crawler_status in failed_status:
						raise Exception(f"Status: {crawler_status}")
				metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
				print("Status: %s", crawler_status)
				print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
				print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
				print("Tables Created: %s", metrics['TablesCreated'])
				print("Tables Updated: %s", metrics['TablesUpdated'])
				print("Tables Deleted: %s", metrics['TablesDeleted'])

				return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)
		else:
			if crawler_state == 'READY':
					print("State: %s", crawler_state)
					print("crawler_config: %s", crawler)
					crawler_status = crawler['Crawler']['LastCrawl']['Status']
					if crawler_status in failed_status:
							raise Exception(f"Status: {crawler_status}")
					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
					print("Status: %s", crawler_status)
					print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
					print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
					print("Tables Created: %s", metrics['TablesCreated'])
					print("Tables Updated: %s", metrics['TablesUpdated'])
					print("Tables Deleted: %s", metrics['TablesDeleted'])

					return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)


CrawlerName = 'crw_fraud_model_input'
crawler_to_run = f'{CrawlerName}'
print('Running Crawler: ', crawler_to_run)
response = wait_for_crawler_completion(crawler_name=crawler_to_run)
print('Finish with status: ', response)

Running Crawler:  crw_fraud_model_input
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
State: %s STOPPING
crawler_config: %s {'Crawler': {'Name': 'crw_fraud_model_input', 'Role': 'sdlf-lakeformation-admin', 'Targets': {'S3Targets': [{'Path': 's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ml_score_aws/', 'Exclusions': []}], 'JdbcTargets': [], 'MongoDBTargets': [], 'DynamoDBTargets': [], 'CatalogTargets': [], 'DeltaTargets'